In [1]:
import pandas as pd

In [4]:
import sys
sys.path.insert(0, "../../../")
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import torch
 
data = pd.read_csv(r"sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

C:\Users\danie\git\GraphAware\time_series\..\dataAnalysis\data\Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']
C:\Users\danie\git\GraphAware\time_series\..\dataAnalysis\data\Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']
C:\Users\danie\git\GraphAware\time_series\..\dataAnalysis\data\Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [3]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
max_Id = data["Id"].unique().max()
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)
data = pd.concat((data, gw_data))
data = data.sort_values(["Id", "Time"])
data = data.reset_index(drop=True)
popped_index = data.pop("index")

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC
0,1,25,W,Control,Greifswald,Validation,AMB,1,0.0,NaN,NaN,15.50,7.0,80.5,NaN,264.0,4.20,8.40
1,2,75,M,Control,Greifswald,Validation,GEN,1,0.0,NaN,NaN,7.40,8.4,87.9,NaN,260.0,4.80,8.47
2,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,0.0,NaN,NaN,96.10,4.8,81.7,NaN,385.0,3.00,13.20
3,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,318840.0,NaN,NaN,57.00,4.4,82.2,NaN,416.0,2.80,14.20
4,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,578640.0,NaN,NaN,93.40,5.7,82.0,0.22,437.0,3.50,13.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709563,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,3978720.0,NaN,NaN,23.95,5.9,95.8,NaN,133.0,2.84,10.00
2709564,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4230480.0,NaN,NaN,19.57,4.6,98.2,NaN,67.0,2.19,3.90
2709565,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4405020.0,NaN,NaN,NaN,6.0,96.6,NaN,110.0,2.91,8.20
2709566,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4489680.0,NaN,NaN,17.41,5.5,98.1,NaN,105.0,2.66,6.70


In [5]:
import numpy as np
from tqdm.notebook import tqdm

source_edge_index = []
target_edge_index = []

for Id, group in tqdm(data.groupby("Id")):
    indices = group.index
    offset = indices[0]
    num_nodes = len(indices)
    edge_index = torch.zeros((2, sum(range(num_nodes + 1))), dtype=torch.long)+offset

    ## Self edges
    edge_index[:, 0:num_nodes] = (torch.arange(num_nodes) + offset).view(1, -1)
    idx = num_nodes
    for i in range(1, num_nodes):
        edge_index[1, idx:idx + i] = i+offset
        edge_index[0, idx:idx + i] = torch.arange(i)+offset
        idx += i
    
    source_edge_index.extend(edge_index[0, :].numpy().tolist())
    target_edge_index.extend(edge_index[1, :].numpy().tolist())

edge_index = np.asarray([np.asarray(source_edge_index), np.asarray(target_edge_index)])
edge_index = torch.tensor(edge_index)

  0%|          | 0/752186 [00:00<?, ?it/s]

In [95]:
data

,Id,Age,Sex,Diagnosis,Center,Set,Sender,Episode,Time,TargetIcu,SecToIcu,CRP,HGB,MCV,PCT,PLT,RBC,WBC
0,1,25,W,Control,Greifswald,Validation,AMB,1,0.0,NaN,NaN,15.50,7.0,80.5,NaN,264.0,4.20,8.40
1,2,75,M,Control,Greifswald,Validation,GEN,1,0.0,NaN,NaN,7.40,8.4,87.9,NaN,260.0,4.80,8.47
2,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,0.0,NaN,NaN,96.10,4.8,81.7,NaN,385.0,3.00,13.20
3,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,318840.0,NaN,NaN,57.00,4.4,82.2,NaN,416.0,2.80,14.20
4,3,77,W,Sepsis,Greifswald,Validation,OTHER,1,578640.0,NaN,NaN,93.40,5.7,82.0,0.22,437.0,3.50,13.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709563,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,3978720.0,NaN,NaN,23.95,5.9,95.8,NaN,133.0,2.84,10.00
2709564,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4230480.0,NaN,NaN,19.57,4.6,98.2,NaN,67.0,2.19,3.90
2709565,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4405020.0,NaN,NaN,NaN,6.0,96.6,NaN,110.0,2.91,8.20
2709566,778980,56,M,Sepsis,Leipzig,Validation,GEN,2,4489680.0,NaN,NaN,17.41,5.5,98.1,NaN,105.0,2.66,6.70


In [10]:
from torch_geometric.utils import add_self_loops

edge_index = add_self_loops(edge_index)[0]

In [92]:
from dataAnalysis.Constants import FEATURES_IN_TABLE, SEX_COLUMN_NAME, SEX_CATEGORY_COLUMN_NAME, LABEL_COLUMN_NAME
## Lets try some edge index based forward padding for one time step -> i.e. single measurements will have the same measurement twice, series will have the measurement and its respective last value
source = edge_index[0]
target = edge_index[1]


In [47]:
X = data[FEATURES_IN_TABLE]
X.loc[:,SEX_CATEGORY_COLUMN_NAME] = (X.loc[:, SEX_COLUMN_NAME] == "W").astype(int)

C:\Users\danie\AppData\Local\Temp\ipykernel_22936\2690060339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:,SEX_CATEGORY_COLUMN_NAME] = (X.loc[:, SEX_COLUMN_NAME] == "W").astype(int)


In [51]:
X = X.drop(SEX_COLUMN_NAME, axis = 1).values

In [94]:
import torch
# X = torch.from_numpy(X)
y = torch.from_numpy(data[LABEL_COLUMN_NAME].values)

KeyError: 'Label'

In [59]:
source_X = torch.index_select(X, 0, source)
target_X = torch.index_select(X, 0, target)

In [83]:
X_time = torch.concat((source_X.unsqueeze(1), target_X.unsqueeze(1)), dim = 1)

In [88]:
from torch import nn
from torch.autograd import Variable 

class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, num_classes) #fully connected 1

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size)) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.shape[0], self.hidden_size)) #internal state
        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) #first Dense
        return out

In [91]:
class SBCData:
    def __init__(self, x_time, y):
        self.x_time = x_time
        self.y = y
    
    def __getitem__(self, i):
        return self.x_time[i], self.y[i]
    
    def __len__(self):
        return len(self.x_time)
dataset = SBCData(X_time, y)

torch.Size([28317462, 2, 7])

In [89]:
model = LSTM1(2, X.shape[1], 16, 1, 2)
optim = torch.optim.Adam(params=model.parameters(), lr = 1e-2)
epochs = 10

## no traon test split now
for epoch in range(epochs):
    model.train()
    logits = model(X_time)
    loss = torch.nn.functional.binary_cross_entropy_with_logits(X, y)
    print(loss.item())
    optim.zero_grad()
    loss.backward()
    optim.step()

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 28997081088 bytes.